In [1]:
import pdb

In [44]:
import os

import numpy as np
import pandas as pd

def fn_getSamples(bPositive, strPart, strJobsDir):
    npNArrTotalSamples = []
    for name in os.listdir(strJobsDir):
        strJobDir = os.path.join(strJobsDir, name)
        npNArrJobSamples = fn_getSamplesFromAJob(bPositive, strPart, strJobDir)
        for npNArr in npNArrJobSamples:
            npNArrTotalSamples.append(npNArr)
    
    npSamples = np.array(npNArrTotalSamples)
    return npSamples
        
        
def fn_getSamplesFromAJob(bPositive, strPart, strJobDir):
    npNArrSamples = []
    strSpecificDir = ""
    if bPositive:
        strSpecificDir = os.path.join(strJobDir, "samples/positive")
    else:
        strSpecificDir = os.path.join(strJobDir, "samples/negtive")
        
    for name in os.listdir(strSpecificDir):
        strStatusDir = os.path.join(strSpecificDir, name)
        strSectionsDir = os.path.join(strStatusDir, "sections")
        for name in os.listdir(strSectionsDir):
            if name == strPart:
                strSectionDir = os.path.join(strSectionsDir, name)
                npNArr = np.load(os.path.join(strSectionDir, "status.npy"))
                npNArrSamples.append(npNArr)
                break
                
    return npNArrSamples

In [49]:
import tensorflow as tf
import tensorflow.keras as keras

In [57]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense
from tensorflow import math

import numpy as np

class GANOperator(object):
    def __init__(self, nLatentDim, nHiddenDimG, nHiddenDimD, nSeqLength, nBatchSize, nFeatures, npNArrSample):
        self.nHiddenDimG = nHiddenDimG
        self.nHiddenDimD = nHiddenDimD
        self.nSeqLength = nSeqLength
        self.nBatchSize = nBatchSize
        self.nFeatures = nFeatures
        
        self.npNArrSample = npNArrSample
        self.oSeqTrainableDiscriminator = fn_makeDiscriminatorTrainable()
        self.oSeqTrainableAdversarial = fn_makeAdversarialTrainable()
        self.oSeqGenerator = fn_makeGenerator()

        
    def fn_makeGenerator(self):
        if self.oSeqGenerator:
            return self.oSeqGenerator
        self.oSeqGenerator = Sequential()
        self.oSeqGenerator.add(GRU(nHiddenDimG, dropout = 0.1, recurrent_dropout = 0.5, return_sequences = True))
        self.oSeqGenerator.add(Dense(nFeatures, activation = math.tanh))
        return self.oSeqGenerator
    
    def fn_makeDiscriminator(self):
        if self.oSeqDiscriminator:
            return self.oSeqDiscriminator
        self.oSeqDiscriminator = Sequential()
        self.oSeqDiscriminator.add(GRU(nHiddenDimD, dropout = 0.1, recurrent_dropout = 0.5, return_sequences = True))
        self.oSeqDiscriminator.add(Dense(nFeatures))
        return self.oSeqDiscriminator
    
    def fn_DLoss(tensorTrue, tensorPred):
        nRow = tensorTrue.shape[0]
        nRow = nRow / 2
        tensorOnes, tensorReal = tensorTrue[:nRow, :], tensorPred[:nRow, :]
        tensorZeros, tensorFake = tensorTrue[nRow:, :], tensorPred[nRow:, :]
        
        tensorLossReal = math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tensorReal, labels = tensorOnes), 1)
        tensorLossFake = math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tensorFake, labels = tensorZeros), 1)
        tensorLoss = tensorLossReal + tensorLossFake
        return math.reduce_mean(tensorLoss)
        
    def fn_GLoss(tensorTrue, tensorPred):
        tensorLoss = math.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tensorPred, labels = tensorTrue), 1)
        return math.reduce_mean(tensorLoss)
    
    def fn_makeDiscriminatorTrainable(self):
        if self.oSeqTrainableDiscriminator:
            return self.oSeqTrainableDiscriminator
        oOptimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.1)
        self.oSeqTrainableDiscriminator = Sequential()
        self.oSeqTrainableDiscriminator.add(self.fn_makeDiscriminator())
        self.oSeqTrainableDiscriminator.compile(loss=fn_DLoss, optimizer=oOptimizer, metrics=["accuracy"])
        return self.oSeqTrainableDiscriminator
    
    def fn_makeAdversarialTrainable(self):
        if self.oSeqTrainableAdversarial:
            return self.oSeqTrainableAdversarial
        oOptimer = tf.optimizers.Adam()
        self.oSeqTrainableAdversarial = Sequential()
        self.oSeqTrainableAdversarial.add(self.fn_makeGenerator())
        self.oSeqTrainableAdversarial.add(self.fn_makeDiscriminator())
        self.oSeqTrainableAdversarial.compile(loss = fn_GLoss, optimizer=oOptimer, metrics=["accuracy"])
        return self.oSeqTrainableAdversarial
    
    def fn_train(self, nEpochs = 2000, nBatchSize = 256):
        for epoch in range(nEpochs):
            npNArrSample = self.npNArrSample[np.random.randint(0, self.npNArrSample.shape[0], size = nBatchSize), :, :]
            npNoise = np.random.uniform(-1.0, 1.0, size = [nBatchSize, nSeqLength, nLatentDim])
            npNArrFake = oSeqGenerator.predict(npNoise)
            npNArrX = np.concatenate((npNArrSample, npNArrFake))
            npNArrY = np.ones([2 * nBatchSize, 1])
            npNArrY[nBatchSize:, :] = 0
            fLossD = oSeqTrainableDiscriminator.train_on_batch(npNArrX, npNArrY)
            
            npNArrY = np.ones([nBatchSize, 1])
            npNoise = np.random.uniform(-1.0, 1.0, size = [nBatchSize, 100])
            fLossA = oSeqTrainableAdversarial.train_on_batch(npNoise, npNArrY)

            strMsg = "%d: [D loss: %f, acc: %f]" % (i, fLossD[0], fLossD[1])
            strMsg = "%s [A loss: %f, acc: %f]" % (strMsg, fLossA[0], fLossA[1])

In [56]:
np.random.uniform(-1.0, 1.0, size = [2, 3, 4])

array([[[ 0.95965273, -0.56959686, -0.66869449,  0.36588037],
        [-0.13474461, -0.00982187,  0.33888038, -0.82310457],
        [ 0.15054734,  0.99423397, -0.22550095,  0.81209248]],

       [[ 0.86594201, -0.5032699 ,  0.65057817, -0.14016409],
        [-0.39339544,  0.33758776, -0.25818414, -0.22497899],
        [ 0.47629576, -0.83481436, -0.0445314 ,  0.66225107]]])

In [ ]:
class operator(object)